In [34]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false').read()
soup = BeautifulSoup(page)
print type(soup)

<class 'bs4.BeautifulSoup'>


In [35]:
soup.prettify()[0:1000]

u'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml">\n <head>\n  <script src="http://sports-ak.espn.go.com/sports/optimizely.js">\n  </script>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any"/>\n  <meta content="#CC0000" name="theme-color"/>\n  <script type="text/javascript">\n   if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {\n        window.location = \'http://a.m.espn.go.com/mobilecache/general/apps/sc\';\n    }\n  </script>\n  <title>\n   2015-16 Regular Season NBA Player Stats and League Leaders - Scoring Per Game - National Basketball Association - ESPN\n  </title>\n  <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>\n  <meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>\n  <meta content="noodp" name="googlebot"/>\n  <meta co

## Parsing HTML

By "parsing HTML", we mean pulling out only the relevant tags/attributes for our analysis.  What Beautiful Soup does is provide a handy bunch of methods for doing this efficiently.

### find method

The find method will search for and return the first tag matching your corresponding search criteria, if it exists.  You can specify tag and attribute info etc.  There is also a findAll method that will return a collection of tags matching your query.

In [36]:
table_div = soup.find(id='my-players-table')
# print table_div.prettify()[0:4000]

Notice that the object returned by the find method is just another inner HTML structure, which we can step through just like we would have with the original soup object.  We've gone to the location in the webpage where the table that we seek starts, now we can use find again to get to the table data.

In [37]:
table = table_div.find("table")
print table

<table cellpadding="3" cellspacing="1" class="tablehead">
<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/play

### Searching by Attributes

Now that we have the table object, we need to step through the rows.  First we'll find the header row so we can populate what the field names will be in our data.  Here we're searching for tags under the table tag whose class attritbute is "colhead".  

In [38]:
table_head = table.find(attrs={"class":'colhead'})

print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
   PTS
  </a>
 </td>
 <td>
  <span title="Field Goals Made-Attempted Per Game">
   FGM-FGA
  </span>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
   FG%
  </a>
 </td>
 <td>
  <span title="3-Point Field Goals Made-Attempted Per Game">
   3PM-3PA
  </span>
 </td>
 <t

Now we find the actual values by searching for the 'td' tags, which is the tag for table data.

In [39]:
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td>, <td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td>, <td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qualified/false" t

Finally, we step through these columns and save them to a list to be used later.  We'll ignore the rank column (RK) because that doesn't give us anything we want later.  We also separate the **PLAYER** column into **PLAYER** and **POSITION**.

In [40]:
cols = []
for header_col in header_cols:
    val = header_col.string
    print val
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

RK
PLAYER
TEAM
GP
MPG
PTS
FGM-FGA
FG%
3PM-3PA
3P%
FTM-FTA
FT%
[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


### Stepping Through a Table

The table rows are indicated by the tag 'tr'.  Again we can find them all and iterate through them.  Within each row we iterate through the respective columns.

In [41]:
table_rows = table.findAll('tr')
# print table_rows

We will save our results in 2 different ways to demonstrate how we can handle both.  The first will be a list of dicts where the key is the field name and the value is the field value.  The second will just be a list of lists of stats with no field name values (we've already defined them earlier).

In [42]:
players_stats_dicts = []
players_stats_array = []
for row in table_rows:
#     print row
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    players_stats_dicts.append(player_stats)
# print players_stats_dicts[0:5]        
# print players_stats_array[0]        


Here we've used the zip function to combine pairs of lists into tuples, and then transformed that into a dict to get a dictionary of FIELD --> VALUE for every player in the table.

Beautiful Soup has many other features, including the ability to step up, down, and sideways in the HTML tree and basically search for any tags, attributes, or values.  For more, take a look at the [Documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## Load into Pandas

Let's load our scraped data into Pandas and take a look at it.  Here is the first way we can do it, simply directly from the dictionary we defined.

In [43]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.475,5.4-11.4,.553,11.1-20.1,.915,6.1-6.7,7,33.3,Stephen Curry,PG,33.9,GS
1,.422,2.7-6.4,.503,10.4-20.7,.885,6.6-7.4,7,37.3,Kevin Durant,SF,30.1,OKC
2,.240,2.6-10.7,.377,8.1-21.6,.854,10.9-12.7,7,39.4,James Harden,SG,29.7,HOU
3,.000,0.0-0.5,.598,11.7-19.5,.783,6.0-7.7,6,35.2,Blake Griffin,PF,29.3,LAC
4,.429,3.9-9.0,.476,9.7-20.4,.871,3.9-4.4,7,36.4,Damian Lillard,PG,27.1,POR


Now here is a 2nd way we can do it.  We convert the 2D stats array into a numpy array and create a Pandas dataframe from it along with the list of column headers we defined earlier.

In [44]:
np_array = np.array(players_stats_array)
df = pd.DataFrame(np_array, columns=cols)
df.head()


,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Stephen Curry,PG,GS,7,33.3,33.9,11.1-20.1,.553,5.4-11.4,.475,6.1-6.7,.915
1,Kevin Durant,SF,OKC,7,37.3,30.1,10.4-20.7,.503,2.7-6.4,.422,6.6-7.4,.885
2,James Harden,SG,HOU,7,39.4,29.7,8.1-21.6,.377,2.6-10.7,.240,10.9-12.7,.854
3,Blake Griffin,PF,LAC,6,35.2,29.3,11.7-19.5,.598,0.0-0.5,.000,6.0-7.7,.783
4,Damian Lillard,PG,POR,7,36.4,27.1,9.7-20.4,.476,3.9-9.0,.429,3.9-4.4,.871


## Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

The general steps should be as follows:
- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)
- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.
- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).
- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result

## Tasks

Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)

In [45]:
def get_cols(table):
    cols = []
    table_head = table.find(attrs={"class":'colhead'})
    for header_col in table_head:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    return cols

 
print get_cols(table)

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.


In [46]:
def get_data(table):
    table_rows = table.findAll('tr')
    players_stats_array = []
    for row in table_rows:
        if row.attrs['class'][0]=='colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')

        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        players_stats_array.append(player_stats)

    return players_stats_array
    
def merge_columns_to_rows(columns, rows):
    players_stats_dicts = []
    for player_stats in rows:
        player_stats = dict(zip(columns, player_stats))
        players_stats_dicts.append(player_stats)
    return players_stats_dicts

rows = get_data(table)
columns = get_cols(table)
print merge_columns_to_rows(columns, rows)


[{u'FGM-FGA': u'11.1-20.1', u'MPG': u'33.3', u'FTM-FTA': u'6.1-6.7', u'FG%': u'.553', u'GP': u'7', u'PLAYER': u'Stephen Curry', u'FT%': u'.915', u'TEAM': u'GS', u'3PM-3PA': u'5.4-11.4', 'POSITION': u'PG', u'PTS': u'33.9', u'3P%': u'.475'}, {u'FGM-FGA': u'10.4-20.7', u'MPG': u'37.3', u'FTM-FTA': u'6.6-7.4', u'FG%': u'.503', u'GP': u'7', u'PLAYER': u'Kevin Durant', u'FT%': u'.885', u'TEAM': u'OKC', u'3PM-3PA': u'2.7-6.4', 'POSITION': u'SF', u'PTS': u'30.1', u'3P%': u'.422'}, {u'FGM-FGA': u'8.1-21.6', u'MPG': u'39.4', u'FTM-FTA': u'10.9-12.7', u'FG%': u'.377', u'GP': u'7', u'PLAYER': u'James Harden', u'FT%': u'.854', u'TEAM': u'HOU', u'3PM-3PA': u'2.6-10.7', 'POSITION': u'SG', u'PTS': u'29.7', u'3P%': u'.240'}, {u'FGM-FGA': u'11.7-19.5', u'MPG': u'35.2', u'FTM-FTA': u'6.0-7.7', u'FG%': u'.598', u'GP': u'6', u'PLAYER': u'Blake Griffin', u'FT%': u'.783', u'TEAM': u'LAC', u'3PM-3PA': u'0.0-0.5', 'POSITION': u'PF', u'PTS': u'29.3', u'3P%': u'.000'}, {u'FGM-FGA': u'9.7-20.4', u'MPG': u'36.4', 

- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).


In [47]:
base_url = 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/'

def all_pages(base_url):
    counter = 1
    results = []
    pages_cols = get_cols(get_nba_stats_table(base_url))
    
    while True:
        current_page = base_url + str(counter)
        page_rows = get_data(get_nba_stats_table(current_page))
        data = merge_columns_to_rows(pages_cols, page_rows)
        if data == []:
            break
        results = results + data
        counter += 40
    return results
    
def get_nba_stats_table(url):
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page)
    table_div = soup.find(id='my-players-table')
    table = table_div.find("table")
    return table

def build_dataframe(url):
    data = all_pages(url)
    return pd.DataFrame.from_dict(data)

r = build_dataframe(base_url)
r.head()


,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.475,5.4-11.4,.553,11.1-20.1,.915,6.1-6.7,7,33.3,Stephen Curry,PG,33.9,GS
1,.422,2.7-6.4,.503,10.4-20.7,.885,6.6-7.4,7,37.3,Kevin Durant,SF,30.1,OKC
2,.240,2.6-10.7,.377,8.1-21.6,.854,10.9-12.7,7,39.4,James Harden,SG,29.7,HOU
3,.000,0.0-0.5,.598,11.7-19.5,.783,6.0-7.7,6,35.2,Blake Griffin,PF,29.3,LAC
4,.429,3.9-9.0,.476,9.7-20.4,.871,3.9-4.4,7,36.4,Damian Lillard,PG,27.1,POR


## - Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them

In [61]:
scoring_url = "http://espn.go.com/nba/statistics/player/_/stat/scoring/qualified/false/count/"

score = build_dataframe(scoring_url)
score.head()


,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.475,38-80,.553,78-141,.915,43-47,7,33.3,Stephen Curry,PG,237,GS
1,.422,19-45,.503,73-145,.885,46-52,7,37.3,Kevin Durant,SF,211,OKC
2,.240,18-75,.377,57-151,.854,76-89,7,39.4,James Harden,SG,208,HOU
3,.429,27-63,.476,68-143,.871,27-31,7,36.4,Damian Lillard,PG,190,POR
4,.314,11-35,.476,70-147,.846,33-39,7,35.7,Russell Westbrook,PG,184,OKC


In [60]:
assists_url = "http://espn.go.com/nba/statistics/player/_/stat/assists/qualified/false/count/"
ass = build_dataframe(assists_url)
ass.head()

,AP48M,APG,AST,AST/TO,GP,MPG,PLAYER,POSITION,TEAM,TO,TOPG
0,14.6,10.9,76,2.24,7,35.7,Russell Westbrook,PG,OKC,34,4.9
1,14.0,9.2,46,4.60,5,31.6,Ricky Rubio,PG,MIN,10,2.0
2,12.2,8.8,53,1.77,6,34.8,John Wall,PG,WSH,30,5.0
3,12.2,8.2,41,2.93,5,32.2,Chris Paul,PG,LAC,14,2.8
4,13.4,7.5,45,4.09,6,26.8,Ish Smith,PG,NO,11,1.8


- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result


In [103]:
ass.set_index(['PLAYER'])
score.set_index(['PLAYER'])
complete = ass.join(score,rsuffix='_r', how = 'outer', on='PLAYER')
complete.head()

,AP48M,APG,AST,AST/TO,GP,MPG,PLAYER,POSITION,TEAM,TO,...,FG%,FGM-FGA,FT%,FTM-FTA,GP_r,MPG_r,PLAYER_r,POSITION_r,PTS,TEAM_r
0,14.6,10.9,76,2.24,7,35.7,Russell Westbrook,PG,OKC,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14.0,9.2,46,4.60,5,31.6,Ricky Rubio,PG,MIN,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12.2,8.8,53,1.77,6,34.8,John Wall,PG,WSH,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.2,8.2,41,2.93,5,32.2,Chris Paul,PG,LAC,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13.4,7.5,45,4.09,6,26.8,Ish Smith,PG,NO,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_a

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


In [82]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [83]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
df_n = pd.DataFrame(raw_data, columns = ['subject_id','test_id'])
df_n


,subject_id,test_id
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16
5,7,14
6,8,15
7,9,1
8,10,61
9,11,16
